In [1]:
from jugaad_trader import Zerodha
import qgrid
from smartapi import SmartConnect
from rich import  print
from smartapi import SmartWebSocket
import ipywidgets
import json
import pandas as pd
import datetime
import threading
import math
import ast 
import http.client
import mimetypes
import telepot
conn = http.client.HTTPSConnection("apiconnect.angelbroking.com")
import time
from dateutil import parser, tz
from tqdm import tqdm
import nsepython
import pyotp
from client_enums import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
####### IMPOTR AUTH FILE AND STOCKS FILE ######
auth_df = pd.read_excel('./auth_info.xlsx')
stocks_df = pd.read_excel('./stocks.xlsx')
global_exchange_type = 'NSE'
# # print(f'{auth_df}')
# print(f'{share_df}')

qgrid.show_grid(stocks_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [3]:
###### AUTHENTICATION FILE DATAFRAME ########
qgrid.show_grid(auth_df[['ac_name','ac_broker','ac_id']],)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [4]:
class one_client_class:
    def __init__(self,ac_broker,ac_name,ac_id,ac_pass,ac_pin,api_key='api_key',
                 totp_key='totp_key',totp_enabled='totp_enabled'
                ):
        '''one_client_class(ac_broker,ac_name,ac_id,ac_pass,ac_pin,api_key='api_key',
                            totp_key='totp_key',totp_enabled='totp_enabled')'''
        self.ac_name = ac_name
        self.ac_id = ac_id
        self.ac_pass = ac_pass
        self.ac_pin = ac_pin
        self.ac_broker = ac_broker
        self.api_key = api_key
        self.totp_key = totp_key
        self.totp_enabled = totp_enabled
        self.funds_equity = 0.00
        self.funds_commodity = 0.00
        
        self.pending_orders_list = []
        self.orders_list = []
        self.holdings_list = []
        self.positions_list = []
        self.gtt_list = []
        self.stock_status = stock_status
        
        self.zerodha_user = ''
        self.zerodha_user_data = ''
        
        self.angel_user = ''
        self.angel_user_data = {}
        self.angel_request_headers = ''
        self.angel_refreshToken = ''
        self.angel_feedToken = ''
            
        orders_list_type = type(self.pending_orders_list)
        holdings_list_type = type(self.holdings_list)
        positions_list_type = type(self.positions_list)

    def do_login(self):
        login_try_counter = 0
        try:
            if(self.ac_broker=='zerodha'):
                self.do_login_zerodha()
                return self.zerodha_user
            elif(self.ac_broker=='angel'):
                self.do_login_angel()
                return self.angel_user
        except Exception as excp:
            if(login_try_counter>=3):
                print('Login Tries exceeded...')
                return
            print(f'Error in login : {excp}')
            time.sleep(1)
#             self.do_login()
            login_try_counter+=1
                
    def get_orders_list(self) -> list:
        got_orders_list = []
        self.pending_orders_list = []
        if(self.ac_broker=='zerodha'):
            got_orders_list = self.zerodha_user.orders()
        elif(self.ac_broker=='angel'):
            got_orders_list = self.angel_user.orderBook()['data']
        if(got_orders_list==None):
            return got_orders_list  
        if(len(got_orders_list)==0):
            return got_orders_list
        for one_order in got_orders_list:
            self.orders_list.append(order_class(one_order))
        return self.orders_list

    def get_pending_orders(self) -> list:
        got_orders_list = []
        self.pending_orders_list = []
        if(self.ac_broker=='zerodha'):
            got_orders_list = self.zerodha_user.orders()
        elif(self.ac_broker=='angel'):
            got_orders_list = self.angel_user.orderBook()['data']
        if(got_orders_list==None):
            return got_orders_list  
        if(len(got_orders_list)==0):
            return got_orders_list
        for one_order in got_orders_list:
            if(one_order['status']=='OPEN' or one_order['status']=='open'):
                self.pending_orders_list.append(order_class(one_order))
                
        return self.pending_orders_list
    
    
    def get_holdings_list(self) -> list:
        got_holding_list = []
        self.holdings_list = []
        if(self.ac_broker=='zerodha'):
            got_holding_list = self.zerodha_user.holdings()
        elif(self.ac_broker=='angel'):
            got_holding_list = self.angel_user.holding()['data']
            if(got_holding_list==None):
                return []
        if(len(got_holding_list)==0):
            return got_holding_list
        for one_holding in got_holding_list:
            self.holdings_list.append(holding_class(one_holding))
        return self.holdings_list    
    
    def get_positions_list(self) -> list:
        got_positions_list = []
        self.positions_list = []
        if(self.ac_broker=='zerodha'):
            got_positions_list = self.zerodha_user.positions()['net']
        elif(self.ac_broker=='angel'):
            got_positions_list = self.angel_user.position()['data']
            if(got_positions_list==None):
                return []
        if(len(got_positions_list)==0 ):
            return got_positions_list
        for one_position in got_positions_list:
            self.positions_list.append(position_class(one_position))
        return self.positions_list
    
    
    def get_gtt_list(self) -> list:
        got_gtt_list = []
        self.gtt_list = []
        if(self.ac_broker=='zerodha'):
            got_gtt_list = self.zerodha_user.get_gtts()
        elif(self.ac_broker=='angel'):
            pass
        for one_gtt in got_gtt_list:
            self.gtt_list.append(zerodha_gtt_status_class(one_gtt))
        return self.gtt_list
        
        
    def check_funds(self):
        if(self.ac_broker=='zerodha'):
            self.funds_equity = math.floor(self.zerodha_user.margins()['equity']['available']['live_balance'])
            self.funds_commodity = math.floor(self.zerodha_user.margins()['commodity']['available']['live_balance'])
        elif(self.ac_broker=='angel'):
            self.funds_equity = math.floor(float(self.angel_user.rmsLimit()['data']['availablecash']))
            self.funds_commodity = math.floor(float(self.angel_user.rmsLimit()['data']['availablecash']))
        
    def do_login_zerodha(self):
        if(self.totp_enabled==1):
            this_totp_class = pyotp.TOTP(self.totp_key)
            this_totp_pin = this_totp_class.now()
#             print(self.totp_key)
#             print(this_totp_pin)
            self.zerodha_user = Zerodha(user_id=self.ac_id,
                                       password=self.ac_pass,
                                       twofa=int(this_totp_pin))  
        else:
            self.zerodha_user = Zerodha(user_id=self.ac_id,
                                       password=self.ac_pass,
                                       twofa=int(self.ac_pin))
        self.zerodha_user.login()
        self.zerodha_user_data = self.zerodha_user.profile()
        str1 = 'LOGIN ZERODHA : '
        print(str1 + self.zerodha_user.profile()['user_name'])
        return self.zerodha_user
                    
    def do_login_angel(self):
        self.angel_user = SmartConnect(api_key=self.api_key)
        self.angel_user_data =self.angel_user.generateSession(self.ac_id,self.ac_pass)
        self.angel_refreshToken = self.angel_user_data['data']['refreshToken']
        self.angel_feedToken=self.angel_user.getfeedToken()
        userProfile= self.angel_user.getProfile(self.angel_refreshToken)
        print (f"LOGIN ANGEL : {userProfile['data']['name']}")
        self.angel_request_headers = {
              'Authorization': f'''Bearer {self.angel_user.access_token}''',
              'Content-Type': 'application/json',
              'Accept': 'application/json',
              'X-UserType': 'USER',
              'X-SourceID': 'WEB',
              'X-ClientLocalIP': self.angel_user.clientLocalIP,
              'X-ClientPublicIP': self.angel_user.clientPublicIP,
              'X-MACAddress': self.angel_user.clientMacAddress,
              'X-PrivateKey': self.api_key
            }
        return self.angel_user

            
class my_clients_group:
    
    def __init__(self,user_df):
        self.user_df = user_df
    
    def do_login_df(self):
        self.user_list = []
        for index,one_zerodha_user_row in self.user_df.iterrows():
            this_name = one_zerodha_user_row['ac_name']
            this_uid = one_zerodha_user_row['ac_id']
            this_pass = one_zerodha_user_row['ac_pass']
            this_pin = one_zerodha_user_row['ac_pin']
            this_broker = one_zerodha_user_row['ac_broker']
            this_api_key = one_zerodha_user_row['api_key']
            this_totp_key = one_zerodha_user_row['totp_key']
            this_totp_enabled = one_zerodha_user_row['totp_enabled']

            globals()[str(this_name)] = one_client_class(this_broker,
                                                      this_name,
                                                      this_uid,
                                                      this_pass,
                                                      this_pin,
                                                   this_api_key,
                                                      this_totp_key,
                                                   this_totp_enabled, )
            globals()[str(this_name)].do_login()
            self.user_list.append(globals()[str(this_name)])
            
class stock_status:
    def __init__(self,one_stock_data=object):
        self.symbol = one_stock_data['symbol']
        self.percent = one_stock_data['percent']
        self.buy_price = one_stock_data['buy']
        self.sl_price = one_stock_data['sl']
        self.target_price = one_stock_data['target']
        self.entry_open = one_stock_data['entry_open']
        self.required_qty = 0
        self.available_qty = 0
        self.holding_qty = 0
        self.position_qty = 0

In [5]:
this_clients_group = my_clients_group(auth_df)

In [6]:
this_clients_group.do_login_df()

In [ ]:
# #Test bloks for my clients class ##

# print(f'{hemang_angel.get_orders_list()=}')
# print(f'{hemang_zerodha.get_orders_list()=}')

# print(f'{hemang_angel.get_pending_orders()=}')
# print(f'{hemang_zerodha.get_pending_orders()=}')

# print(f'{hemang_angel.get_holdings_list()=}')
# print(f'{hemang_zerodha.get_holdings_list()=}')

# print(f'{hemang_angel.get_positions_list()=}')
# print(f'{hemang_zerodha.get_positions_list()=}')

In [ ]:
def keep_track():
    try:
        while (True):
            time.sleep(0.1)
            stocks_df = pd.read_excel('./stocks.xlsx')
            symbol_request_list = []
            if(len(stocks_df.symbol.to_list())==0):
                print('STOCK_DF IS EMPTY!')
                continue

            for onestk in stocks_df.symbol:
                symbol_request_list.append(global_exchange_type+':' +onestk)
                print( global_exchange_type+':' +onestk)
                
            this_user_one = this_clients_group.user_list[0]
            quote_map = this_user_one.zerodha_user.quote(symbol_request_list)
            for __,one_stock_row in stocks_df.iterrows():
                for one_user in this_clients_group.user_list:
                    print(one_user.ac_name)
                    this_holding_list = one_user.get_holdings_list()
                    this_position_list  =one_user.get_positions_list()
                    this_stock_status = one_user.stock_status(one_stock_row)
                    one_user.check_funds()
                    required_fund_in_this_stock_for_this_user = (one_user.funds_equity)*(this_stock_status.percent/100)
                    this_ltp = zerodha_quote_class(quote_map[global_exchange_type+':'+this_stock_status.symbol]).last_price
                    this_stock_status.required_qty = int((required_fund_in_this_stock_for_this_user)/this_ltp)
                    for one_holding in this_holding_list:
                        if(one_holding.tradingsymbol==this_stock_status.symbol):
                            this_stock_status.holding_qty += int(one_holding.t1_quantity + one_holding.quantity)

                    for one_position in this_position_list:
                        if(one_position.tradingsymbol==this_stock_status.symbol):
                            this_stock_status.position_qty += int(one_position.quantity)
                    this_stock_status.available_qty += int(this_stock_status.position_qty+this_stock_status.holding_qty)
                    this_gtts_list = one_user.zerodha_user.get_gtts()
                    is_required_gtt_available = False
                    if(len(this_gtts_list)!=0):
                        for one_gtt in this_gtts_list:
                            this_gtt_object = zerodha_gtt_status_class(one_gtt)
                            if(this_gtt_object.condition.tradingsymbol==this_stock_status.symbol):
                                if(this_stock_status.available_qty!=0 
                                   and this_gtt_object.orders.transaction_type_1=='SELL'
                                   and this_gtt_object.orders.price_1==this_stock_status.sl_price
                                   and this_gtt_object.status==one_user.zerodha_user.GTT_STATUS_ACTIVE
                                  ):
                                    is_required_gtt_available = True
                                elif(this_stock_status.available_qty==0
                                     and this_gtt_object.orders.transaction_type_1=='BUY'
                                     and this_gtt_object.orders.price_1==this_stock_status.buy_price
                                     and this_gtt_object.status==one_user.zerodha_user.GTT_STATUS_ACTIVE
                                    ):
                                    is_required_gtt_available = True
                            if((this_gtt_object.condition.tradingsymbol not in (stocks_df.symbol.to_list()))
                               and (this_gtt_object.status==one_user.zerodha_user.GTT_STATUS_ACTIVE)
                              ):
                                print(this_gtt_object.condition.tradingsymbol)
                                print(stocks_df.symbol)
                                one_user.zerodha_user.delete_gtt(this_gtt_object.id)
                    print(this_stock_status.available_qty)
                    
                    
                    if((this_stock_status.available_qty==0 and this_stock_status.entry_open==1) 
                       and (not is_required_gtt_available)
                       and (this_stock_status.required_qty>0)):
                        one_user.zerodha_user.place_gtt(
                            one_user.zerodha_user.GTT_TYPE_SINGLE,
                            this_stock_status.symbol,
                            global_exchange_type,
                            [ this_stock_status.buy_price],
                            this_ltp,
                            [{'exchange': global_exchange_type,
                              'tradingsymbol': this_stock_status.symbol,
                              'product': 'CNC',
                              'order_type': 'MARKET',
                              'transaction_type': 'BUY',
                              'quantity': this_stock_status.required_qty,
                              'price': this_stock_status.buy_price,
                             },
                            ],)

                    if((this_stock_status.available_qty>0) 
                       and (not is_required_gtt_available)):
                        one_user.zerodha_user.place_gtt(
                            one_user.zerodha_user.GTT_TYPE_OCO,
                            this_stock_status.symbol,
                            global_exchange_type,
                            [this_stock_status.sl_price,
                             this_stock_status.target_price,
                             ],
                            (this_stock_status.sl_price+this_stock_status.target_price)/2,
                            [
                            {'exchange': global_exchange_type,
                              'tradingsymbol': this_stock_status.symbol,
                              'product': 'CNC',
                              'order_type': 'MARKET',
                              'transaction_type': 'SELL',
                              'quantity': this_stock_status.available_qty,
                              'price': this_stock_status.sl_price,
                             },

                            {'exchange': global_exchange_type,
                              'tradingsymbol': this_stock_status.symbol,
                              'product': 'CNC',
                             'order_type': 'MARKET',
                              'transaction_type': 'SELL',
                              'quantity': this_stock_status.available_qty,
                              'price': this_stock_status.target_price,
                             },
                            ],)

                    print(this_stock_status.symbol)
                    print(f'{this_stock_status.required_qty=}')
                    print(f'{this_stock_status.available_qty=}')
                    print('*'*50)
            
    except Exception as cre:
        print(f'{cre}=')
#         this_clients_group.do_login_df()
        keep_track()

In [ ]:
keep_track()

In [9]:
def get_all_stats():
    for one_user in this_clients_group.user_list:
        this_holding_list = one_user.get_holdings_list()
        this_position_list = one_user.get_positions_list()
        this_gtt_list  =one_user.get_gtt_list()    

    positions_list_list_json =  ([([{'user':y.ac_name,'pos':x.__dict__} for x in y.positions_list ]) for y in this_clients_group.user_list])
    holdings_list_list_json =  ([([{'user':y.ac_name,'hld':x.__dict__} for x in y.holdings_list ]) for y in this_clients_group.user_list])
    gtts_list_list_json = ([([{'user':y.ac_name,'gtt':x.__dict__} for x in y.gtt_list ]) for y in this_clients_group.user_list])

    total_holdings_list = []
    for one_user_holding in holdings_list_list_json:
        for one_holding_ip in one_user_holding:
            total_holdings_list.append(flatten_json(one_holding_ip))


    total_positions_list = []
    for one_user_position in positions_list_list_json:
        for one_position_ip in one_user_position:
            total_positions_list.append(flatten_json(one_position_ip))


    total_gtt_list = []
    for one_user_gtt in gtts_list_list_json:
        for one_gtt_ip in one_user_gtt:
            total_gtt_list.append(flatten_json(one_gtt_ip))

    gtt_dataframe = pd.DataFrame(total_gtt_list)
#     display(gtt_dataframe)
    position_dataframe = pd.DataFrame(total_positions_list)
#     display(position_dataframe)    
    holding_dataframe = pd.DataFrame(total_holdings_list)
#     display(holding_dataframe)   

    holding_dataframe.to_excel('./plot_df/holding.xlsx',index=False)
    position_dataframe.to_excel('./plot_df/positions.xlsx',index=False)
    gtt_dataframe.to_excel('./plot_df/gtts.xlsx',index=False)


In [11]:
while True:
    get_all_stats()

TokenException: Incorrect `api_key` or `access_token`.

In [ ]:
# ########## OCO GTT SAMPLE ########### LOADED !!! DO NOT EXECUTE !!!!!   ########
# hemang_zerodha.zerodha_user.place_gtt(
#     hemang_zerodha.zerodha_user.GTT_TYPE_OCO,
                                      
#     'NIFTY21JULFUT',
                                      
#     hemang_zerodha.zerodha_user.EXCHANGE_NFO,
                                      
#                                     [ 15540.9,16175.25,],
#                                       16000.25,
#                                        [{'exchange': 'NFO',
#                                         'tradingsymbol': 'NIFTY21JULFUT',
#                                         'product': 'NRML',
#                                         'order_type': 'LIMIT',
#                                         'transaction_type': 'BUY',
#                                         'quantity': 50,
#                                         'price': 15540.95,
#                                         'result': None},
#                                        {'exchange': 'NFO',
#                                         'tradingsymbol': 'NIFTY21JULFUT',
#                                         'product': 'NRML',
#                                         'order_type': 'LIMIT',
#                                         'transaction_type': 'BUY',
#                                         'quantity': 50,
#                                         'price': 16175.0,
#                                         'result': None}],)
# ################################################################################

# ########## OCO GTT SAMPLE ########### LOADED !!! DO NOT EXECUTE !!!!!   ########
# hemang_zerodha.zerodha_user.place_gtt(
#     hemang_zerodha.zerodha_user.GTT_TYPE_SINGLE,
#     'NIFTY21JULFUT',
#     hemang_zerodha.zerodha_user.EXCHANGE_NFO,
#                                     [ 428.00,],
#                                       431.05,
#                                        [{'exchange': 'NSE',
#                                         'tradingsymbol': 'SBIN',
#                                         'product': 'NRML',
#                                         'order_type': 'MARKET',
#                                         'transaction_type': 'BUY',
#                                         'quantity': 50,
#                                         'price': 428.00,
#                                         'result': None},],)
# ################################################################################
# # [{'id': 50600729,
# #   'user_id': 'FC5917',
# #   'parent_trigger': None,
# #   'type': 'two-leg',
# #   'created_at': '2021-07-14 21:06:21',
# #   'updated_at': '2021-07-14 21:06:21',
# #   'expires_at': '2021-07-30 00:00:00',
# #   'status': 'active',
# #   'condition': {'exchange': 'NFO',
# #    'last_price': 15858.1,
# #    'tradingsymbol': 'NIFTY21JULFUT',
# #    'trigger_values': [15540.9, 16175.25],
# #    'instrument_token': 13614338},
# #   'orders': [{'exchange': 'NFO',
# #     'tradingsymbol': 'NIFTY21JULFUT',
# #     'product': 'NRML',
# #     'order_type': 'LIMIT',
# #     'transaction_type': 'SELL',
# #     'quantity': 50,
# #     'price': 15540.95,
# #     'result': None},
# #    {'exchange': 'NFO',
# #     'tradingsymbol': 'NIFTY21JULFUT',
# #     'product': 'NRML',
# #     'order_type': 'LIMIT',
# #     'transaction_type': 'SELL',
# #     'quantity': 50,
# #     'price': 16175.5,
# #     'result': None}],
# #   'meta': {}}]
# ################################################################################

In [ ]:
st.dataframe()